## Yolo11 tests

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os , sys
sys.path.append('..')
from pathlib import Path
cur_path = Path(os.getcwd()).parent
sam2_path = cur_path / 'sam2_octron'
sys.path.append(cur_path.as_posix())
from matplotlib import pyplot as plt
import cmasher as cmr
import numpy as np
import seaborn as sns
sns.set_theme(style='white')
%config InlineBackend.figure_format = 'retina'

In [ ]:
from octron.yolo_octron.yolo_octron import YOLO_octron
from octron.yolo_octron.helpers.yolo_checks import check_yolo_models
from octron.yolo_octron.helpers.training import collect_labels

In [ ]:
base_path = Path('/Users/horst/Documents/python/OCTRON/octron/notebooks').parent
models_yaml_path = base_path / 'yolo_octron/yolo_models.yaml'

# project_path = Path('/Users/horst/Downloads/octron_project_2')
# assert project_path.exists()

### Organize training data 

In [ ]:
# Run collect_labels in isolation. This would be triggered if a new project path is chosen
#label_dict = collect_labels(project_path)   
#label_dict

In [ ]:
# yolo_octron = YOLO_octron(models_yaml_path=models_yaml_path,
#                           )

In [ ]:
# yolo_octron.project_path = project_path

In [ ]:
# from ultralytics import settings

In [ ]:
# yolo_octron.prepare_labels(
#                     prune_empty_labels=True, 
#                     min_num_frames=10, 
#                     verbose=False, 
# )
# prepare_polygons = yolo_octron.prepare_polygons()
# # Process all yielded values
# for no_entry, total_label_dict, label, frame_no, total_frames in prepare_polygons:
#     # add progress reporting here 
#     pass
    
# print("Polygon preparation complete!")
# yolo_octron.prepare_split(
#                     training_fraction=0.7,
#                     validation_fraction=0.15,
#                     verbose=False,
# )

In [ ]:
# create_training_data = yolo_octron.create_training_data()
# # Process all yielded values
# for no_entry, total_label_dict, label, split, frame_no, total_frames in create_training_data:
#     # add progress reporting here 
#     pass
    
# yolo_octron.write_yolo_config()

## YOLO training

In [ ]:
# _ = yolo_octron.load_model('YOLO11l')

In [ ]:
# yolo_octron.launch_tensorboard()

In [ ]:
# train_results = yolo_octron.train(epochs=1)

In [ ]:
# # Load a model
# import time

# path_to_model = Path('/Users/horst/Documents/python/OCTRON/octron/yolo_octron/models/yolo11l-seg.pt')
# config_path = Path('/Users/horst/Downloads/octron_project_2/model/training_data/yolo_config.yaml')
# path_to_training = Path('/Users/horst/Downloads/octron_project_2/model/training_data')

# def on_train_start(trainer):
#     print("🥳  Training is starting!")

# def on_fit_epoch_end(trainer):
#     current_epoch = trainer.epoch + 1 
#     time_epoch = trainer.epoch_time
#     print(f"Time for epoch: {time_epoch}")
#     remaining_time = time_epoch * (no_epochs - current_epoch)   
#     finish_time = time.time() + remaining_time
#     finish_time_str = ' '.join(time.ctime(finish_time).split()[:-1])
#     print(f"Estimated time remaining: {remaining_time} seconds")    
#     print(f'Estimated finish time: {finish_time_str}')  
    
# from ultralytics import YOLO
# model = YOLO(path_to_model)  # load a pretrained model (recommended for training)
# model.add_callback("on_train_start", on_train_start)
# model.add_callback("on_fit_epoch_end", on_fit_epoch_end)
# no_epochs = 1

# # Train the model
# # https://docs.ultralytics.com/usage/cfg/#solutions-settings
# results = model.train(data=config_path, 
#                       save_dir=path_to_training.as_posix(),
#                       name='training',
#                       mode='segment',
#                       device='cpu',
#                       mask_ratio=4,
#                       epochs=no_epochs,
#                       imgsz=640,
#                       resume=False,
#                       plots=True,
#                       batch=.9,
#                       cache=False,
#                       save=True,
#                       save_period=15,
#                       project=None,
#                       exist_ok=True,
#                       # augmentation
#                       augment=True,
#                       hsv_v=.25,
#                       degrees=180,
#                       scale=.5,
#                       shear=2,
#                       flipud=.1,
#                       fliplr=.1,
#                       mosaic=1.0,
#                       copy_paste=.5,
#                       copy_paste_mode='mixup', 
#                       erasing=.25,
#                       crop_fraction=1.0,
#                       )

In [ ]:
project_path = Path('/Users/horst/Downloads/octron_project_2_trained')

In [ ]:
from natsort import natsorted   
import pandas as pd
from datetime import datetime

In [ ]:
from ultralytics import YOLO
from octron.sam2_octron.helpers.video_loader import probe_video, get_vfile_hash
from octron.yolo_octron.helpers.polygons import get_polygons, polygon_to_mask, find_objects_in_mask
from octron.yolo_octron.helpers.yolo_zarr import create_prediction_store, create_prediction_zarr

In [ ]:
yolo_octron = YOLO_octron(models_yaml_path=models_yaml_path)

### Setup video, zarr store and model 

In [ ]:
# Napari PyAV reader 
from napari_pyav._reader import FastVideoReader
import napari
from collections import defaultdict
import cv2
import zarr
from tqdm.auto import tqdm
from octron.sam2_octron.helpers.sam2_zarr import create_image_zarr


In [ ]:
config_path = Path('/Users/horst/Downloads/octron_project_2_trained/model/training_data/yolo_config.yaml')
tracker_yaml_path = config_path.parent / 'tracker_config.yaml'
yolo_octron.write_byte_tracker_yaml(tracker_yaml_path)   

video_for_prediction = '/Users/horst/Downloads/octron_project_2/for prediction/8_behaviour_filtered2024-11-04T14_20_34_20240930_Th19.mp4'
video_path = Path(video_for_prediction)
video_dict = probe_video(video_path)
# Create hash and save it in the metadata
video_file_hash = get_vfile_hash(video_path)
video_dict['hash'] = video_file_hash # Save it locally as metadata
# Layer name 
layer_name = f'VIDEO [name: {video_path.stem}]'
layer_dict = {'name'    : layer_name,
              'metadata': video_dict,
              }

In [18]:
found_models_project = natsorted(project_path.rglob('*training/weights/*.pt'))
print(found_models_project)
print(f'Loading {found_models_project[-1]}')

# Load the YOLO11 model
model = YOLO(found_models_project[-1])

[PosixPath('/Users/horst/Downloads/octron_project_2_trained/model/segment/training/weights/best.pt'), PosixPath('/Users/horst/Downloads/octron_project_2_trained/model/segment/training/weights/epoch0.pt'), PosixPath('/Users/horst/Downloads/octron_project_2_trained/model/segment/training/weights/epoch20.pt'), PosixPath('/Users/horst/Downloads/octron_project_2_trained/model/segment/training/weights/epoch40.pt'), PosixPath('/Users/horst/Downloads/octron_project_2_trained/model/segment/training/weights/epoch60.pt'), PosixPath('/Users/horst/Downloads/octron_project_2_trained/model/segment/training/weights/last.pt')]
Loading /Users/horst/Downloads/octron_project_2_trained/model/segment/training/weights/last.pt


In [ ]:
video = FastVideoReader(video_for_prediction, read_format='rgb24')

### Prepare prediction

In [16]:
save_dir = Path(video_for_prediction.replace('.mp4', '_predictions'))
os.makedirs(save_dir, exist_ok=True)
zarr_store_dir = save_dir / 'predictions.zarr'  # Zarr store directory

In [17]:
# Create initial zarr store reference
prediction_store = create_prediction_store(zarr_store_dir)
# Delete previous predictions
for file in save_dir.rglob('*'):
    os.remove(file)
tracking_df = create_tracking_dataframe(video_dict)
tracking_df.attrs

NameError: name 'create_tracking_dataframe' is not defined

In [ ]:
root = zarr.open_group(store=prediction_store, mode='a')
print("Existing keys in zarr archive:", list(root.array_keys()))

In [ ]:
zarr_root = zarr.open_group(store=prediction_store, mode='a')
poly_smooth_sigma = 1.5
tracking_df_dict = {}
for frame_no, frame in enumerate(tqdm(video, 
                                      total=video_dict['num_frames'], 
                                      desc='Processing video')
                                 ):  
    # Run YOLO11 tracking on the frame, persisting tracks between frames
    results = model.track(source=frame, 
                          persist=True,
                          task='segment',   
                          tracker=tracker_yaml_path.as_posix(),
                          project=save_dir.parent.as_posix(),
                          name=save_dir.name,
                          show=False,
                          save=False,   
                          conf=.75,    
                          iou=.8,
                          device='cpu', 
                          half=False,
                          retina_masks=True,
                          show_boxes=False,
                          save_txt=False,
                          save_conf=False,   
                          )
    
    # Get the boxes and track IDs
    boxes = results[0].boxes.xywh.cpu()
    confidences = results[0].boxes.conf.cpu().numpy()
    label_names = tuple([results[0].names[int(r)] for r in results[0].boxes.cls.cpu().numpy()])
    
    masks_polys = results[0].masks.xy  
    track_ids = results[0].boxes.id.int().cpu().tolist()
    
    # Extract tracks 
    for label, box, mask_poly, track_id, conf in zip(label_names, 
                                                boxes, 
                                                masks_polys, 
                                                track_ids, 
                                                confidences
                                                ):
        # Initialize mask zarr array if needed
        if f'{track_id}_masks' not in list(zarr_root.array_keys()):
            video_shape = (video_dict['num_frames'], video_dict['height'], video_dict['width'])   
            mask_store = create_prediction_zarr(prediction_store, 
                            f'{track_id}_masks',
                            shape=video_shape,
                            chunk_size=50,     
                            fill_value=-1,
                            dtype='int8',                           
                            video_hash=video_file_hash
                            )
            mask_store.attrs['label'] = label
            mask_store.attrs['classes'] = results[0].names
        else:
            mask_store = zarr_root[f'{track_id}_masks']
        # Initialize dataframe if needed
        if track_id not in tracking_df_dict:
            tracking_df = create_tracking_dataframe(video_dict)
            tracking_df.attrs['label'] = label
            tracking_df.attrs['track_id'] = track_id
            # Add to the dictionary
            tracking_df_dict[track_id] = tracking_df
        else:
            tracking_df = tracking_df_dict[track_id]
            assert tracking_df.attrs['track_id'] == track_id, "ID mismatch" 
            assert tracking_df.attrs['label'] == label, "Label mismatch"    
        
        # Masks
        dummy_mask = np.zeros((video_dict['height'], video_dict['width']))    
        mask = polygon_to_mask(dummy_mask.astype('int8'), 
                               mask_poly, 
                               smooth_sigma=poly_smooth_sigma
                               )
        mask_store[frame_no,:,:] = mask
        
        # Get region properties and save them to the dataframe
        labels, regions_props = find_objects_in_mask(mask, min_area=0)
        assert len(regions_props) == 1
        region_prop = regions_props[0]
        # ... extract all the properties we want to track
        centroid = region_prop.centroid
        area = region_prop.area
        eccentricity = region_prop.eccentricity
        orientation = region_prop.orientation
        solidity = region_prop.solidity 
        
        # Store data in DataFrame with flat column names
        tracking_df.loc[(frame_no, track_id), 'pos_x'] = centroid[1]  # x coordinate
        tracking_df.loc[(frame_no, track_id), 'pos_y'] = centroid[0]  # y coordinate
        tracking_df.loc[(frame_no, track_id), 'area'] = area
        tracking_df.loc[(frame_no, track_id), 'eccentricity'] = eccentricity
        tracking_df.loc[(frame_no, track_id), 'orientation'] = orientation
        tracking_df.loc[(frame_no, track_id), 'confidence'] = conf
        tracking_df.loc[(frame_no, track_id), 'solidity'] = solidity

# Save each tracking DataFrame with a label column added
for track_id, tr_df in tracking_df_dict.items():
    label = tr_df.attrs["label"]
    df_to_save = tr_df.copy()
    # Add the label column (will be filled with the same value for all rows)
    df_to_save.insert(0, 'label', label)
    # Save to CSV 
    filename = f'{label}_track_{track_id}.csv'
    csv_path = save_dir / filename
    df_to_save.to_csv(csv_path)
    print(f"Saved tracking data for '{label}' (track ID: {track_id}) to {filename}")

### Read folder of tracking results back in

In [ ]:
import zarr

In [ ]:
csvs = list(save_dir.rglob('*.csv'))
print(f"Found {len(csvs)} tracking CSV files")
zarr_files = list(save_dir.rglob('*.zarr'))
assert len(zarr_files) == 1, "Expected exactly one zarr file"
zarr_file = zarr_files[0]
print(f"Found zarr file: {zarr_file}")

In [ ]:
# Load zarr
store = zarr.storage.LocalStore(zarr_file, read_only=False)
root = zarr.open_group(store=store, mode='a')
print("Existing keys in zarr archive:", list(root.array_keys()))

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(6, 6))
# dummy_mask=np.zeros((video_dict['height'], video_dict['width']))
# ax.imshow(dummy_mask, cmap='gray')  
# plot_every_n_mask = 11
def find_class_by_name(classes, class_name):
    return (next((k for k, v in classes.items() if v == class_name), None))

# Color handling
from napari.utils import DirectLabelColormap
from octron.sam2_octron.helpers.sam2_colors import create_label_colors, sample_maximally_different
all_labels_submaps = create_label_colors(n_labels=10,
                                         n_colors_submap=50
                                         )
indices_max_diff_labels = sample_maximally_different(list(range(10)))
indices_max_diff_subcolors = sample_maximally_different(list(range(50)))


track_df_dict = {}
color_dict = {}
label_trackid_dict = {}
label_counter_dict = {} # This is to find the right color in all_labels_submaps

# Load the tracking data
for csv in save_dir.glob('*.csv'):
    track_df = pd.read_csv(csv)
    assert len(track_df.label.unique()) == 1, "Multiple labels found in tracking data"  
    assert len(track_df.track_id.unique()) == 1, "Multiple track_ids found in tracking data"  
    label = track_df.iloc[0].label
    track_id = track_df.iloc[0].track_id
    label_trackid_dict[label] = track_id
    
    # Check zarr
    assert f'{track_id}_masks' in list(root.array_keys()), "Mask not found in zarr archive" 
    mask_zarr = root[f'{track_id}_masks']
    classes = mask_zarr.attrs.get('classes', None) # These are the original classes from the model
    original_class_id = find_class_by_name(classes,label)
    # Check if an index was already given for the current label
    # This is to extract the right color from the colormap
    if label not in label_counter_dict:
        label_counter_dict[label] = {'track_counter' : 0}
    else:
        label_counter_dict[label]['track_counter'] += 1
    # Find color 
    obj_color =  all_labels_submaps[indices_max_diff_labels[int(original_class_id)]][indices_max_diff_subcolors[label_counter_dict[label]['track_counter']]]
    
    mask_colors = DirectLabelColormap(color_dict={None: [0.,0.,0.,0.], 1: obj_color}, 
                                              use_selection=True, 
                                              selection=1,
                                              )
        
        
    color_dict[int(track_id)] = mask_colors
    track_df_dict[int(track_id)] = track_df
    assert f'{track_id}_masks' in list(root.array_keys()), "Mask not found in zarr archive" 
    mask_zarr = root[f'{track_id}_masks']
    
#     #Plot the track on the dummy mask
    
#     ax.scatter(track_df['pos_x'], 
#                track_df['pos_y'], 
#                s=track_df['area']/400, 
#                label=label, 
#                alpha=.6
#                )
#     track_outline = np.zeros_like(dummy_mask)
#     for frame_no in range(0, video_dict['num_frames'], plot_every_n_mask):
#         mask = mask_zarr[frame_no,:,:]
#         if mask[0,0] == -1:
#             continue
        
#         polygon_points = get_polygons(mask)
#          # Check if polygon_points is valid
#         if polygon_points is None or len(polygon_points) == 0:
#             continue
            
#         # Ensure polygon_points is properly formatted for cv2.polylines
#         if isinstance(polygon_points, np.ndarray):
#             # Convert to list of numpy arrays if it's a single array
#             if len(polygon_points.shape) == 2:  # Single polygon
#                 polygon_points = [polygon_points]
            
#             # Draw each polygon
#             for poly in polygon_points:
#                 # Ensure the points are properly shaped and have integer coordinates
#                 if len(poly) > 2:  # Need at least 3 points to form a polygon
#                     poly_reshaped = poly.reshape((-1, 1, 2)).astype(np.int32)
#                     try:
#                         track_outline = cv2.polylines(
#                             img=track_outline.copy(), 
#                             pts=[poly_reshaped],
#                             isClosed=True, 
#                             color=(255, 255, 255),  
#                             thickness=2,
#                         )
#                     except Exception as e:
#                         print(f"Error with polygon: {e}")
#                         print(f"Polygon shape: {poly.shape}, dtype: {poly.dtype}")
    
#     # Add the track outline to the main plot
#     if np.any(track_outline > 0):
#         ax.imshow(track_outline, cmap='gray', alpha=0.3)
    
# ax.legend()
# ax.set_xticks([]);
# ax.set_yticks([]);
    

### Add predictions back to napari 

In [ ]:
from scipy.ndimage import gaussian_filter1d

In [ ]:
viewer = napari.Viewer()    

add_layer = getattr(viewer, "add_image")
add_layer(video, **layer_dict)

In [ ]:
sigmal_tracking_pos = 1
min_frame = 0
max_frame = video_dict['num_frames'] 

# Loop over each track and add it to the viewer
for track_id_to_plot in track_df_dict.keys():
    
    label = track_df_dict[track_id_to_plot].iloc[0].label   

    track_df_napari = track_df_dict[track_id_to_plot][['track_id','frame','pos_y','pos_x']].copy()
    features_df_napari = track_df_dict[track_id_to_plot][['frame','confidence','area','eccentricity','orientation','solidity']].copy()

    check_continuous = np.all(np.diff(track_df_napari['frame']) == 1)   
    if not check_continuous:
        print("Frames are not continuous ... interpolating")
        complete_tracking   = pd.DataFrame({'frame': range(int(min_frame), int(max_frame))})
        complete_features   = pd.DataFrame({'frame': range(int(min_frame), int(max_frame))})
        # Merge with existing data to identify missing frames
        merged_df_tracking = complete_tracking.merge(track_df_napari, on='frame', how='left')
        merged_df_tracking['track_id'] = track_id_to_plot
        merged_df_features = complete_features.merge(features_df_napari, on='frame', how='left')
        merged_df_features.fillna(0, inplace=True)
        # Make sure frame and track_id are the right types for interpolation
        merged_df_tracking['frame'] = merged_df_tracking['frame'].astype(int)
        merged_df_tracking['track_id'] = merged_df_tracking['track_id'].astype(int)

        # Interpolate the position columns
        pos_cols = ['pos_x', 'pos_y']
        for col in pos_cols:
            merged_df_tracking[col] = merged_df_tracking[col].interpolate(method='linear')

        # # For any remaining NaN values at the start or end, use forward/backward fill
        merged_df_tracking = merged_df_tracking.ffill()
        merged_df_tracking = merged_df_tracking.bfill()
        
        track_df_napari = merged_df_tracking.copy()
        features_df_napari = merged_df_features.copy()
        
    # Smooth the positions
    pos_cols = ['pos_x', 'pos_y']
    for col in pos_cols:
        track_df_napari[col] = gaussian_filter1d(track_df_napari[col], sigma=sigmal_tracking_pos)
        
    for col in features_df_napari.columns:
        features_df_napari[col] = features_df_napari[col].astype(float)
    features_dict = features_df_napari.to_dict(orient='list')
    viewer.add_tracks(track_df_napari.values, 
                      features=features_dict,
                       name=f'{label} - id {track_id_to_plot}', 
                      colormap='hsv',
                )
    viewer.layers[f'{label} - id {track_id_to_plot}'].tail_width = 5
    viewer.layers[f'{label} - id {track_id_to_plot}'].tail_length = 50
    viewer.layers[f'{label} - id {track_id_to_plot}'].color_by = 'frame'
    # Add masks
    mask_zarr = root[f'{track_id_to_plot}_masks']
    labels_layer = viewer.add_labels(
        mask_zarr,
        name=f'{label} - MASKS - id {track_id_to_plot}',  
        opacity=0.1,
        blending='additive',  
        colormap=color_dict[track_id_to_plot], 
    )
    
    viewer.layers[f'{label} - id {track_id_to_plot}'].tail_width = 4
    viewer.layers[f'{label} - id {track_id_to_plot}'].tail_length = 50